In [1]:
import os
import re
from typing import *

import pandas as pd
from transformers import AutoTokenizer
from rank_bm25 import BM25Okapi


class DocumentLoader:
    def __init__(self, dataframe: pd.DataFrame = None) -> None:
        self.dataframe = dataframe
        
        
    def load_documents(self, dataframe: pd.DataFrame = None) -> List[str]:
        dataframe = self.dataframe if dataframe is None else dataframe
        documents = dataframe['범죄 사실'].tolist()
        return documents


class LongformerTokenizer:
    def __init__(self, model_name: str = 'severinsimmler/xlm-roberta-longformer-base-16384') -> None:
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def tokenize(self, text: str) -> List[str]:
        tokens = self.tokenizer.tokenize(text)
        return tokens


class BM25Retriever:
    def __init__(self, documents: List[str], tokenizer: LongformerTokenizer) -> None:
        self.tokenizer = tokenizer
        self.documents = documents
        self.tokenized_documents = [self.tokenizer.tokenize(doc) for doc in documents]
        self.bm25 = BM25Okapi(self.tokenized_documents)
        self.search_history: List[Dict[str, any]] = []


    def retrieve(self, query: str, top_n: int = 5) -> Tuple[List[int], List[float]]:
        tokenized_query = self.tokenizer.tokenize(query)
        bm25_scores = self.bm25.get_scores(tokenized_query)
        sorted_indices = bm25_scores.argsort()[::-1][:top_n]
        
        # Store the search results
        self._store_search_results(query, sorted_indices, bm25_scores)

        return sorted_indices.tolist(), bm25_scores.tolist()


    def _store_search_results(self, query: str, indices: List[int], scores: List[float]) -> None:
        result = {
            "query": query,
            "results": [{"index": idx, 
                         "document": self.documents[idx], 
                         "score": scores[idx]} 
                        for idx in indices]
        }
        self.search_history.append(result)


    def get_search_history(self) -> List[Dict[str, any]]:
        return self.search_history
    
    def save_search_history_to_csv(self, file_path: str = './search_history.csv') -> None:
        records = []
        for entry in self.search_history:
            query = entry["query"]
            for result in entry["results"]:
                record = {
                    "query": query,
                    "document_index": result["index"],
                    "document": " ".join(result["document"]),
                    "score": result["score"]
                }
                records.append(record)
        df = pd.DataFrame(records)
        df.to_csv(file_path, index=False, encoding='utf-8')
        return None

c:\Users\dieyo\OneDrive\1_Project\RAG-Proejct_0530_0605\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
import pandas as pd
df = pd.read_csv("./docs/test_docs.csv")
sample = df.iloc[4]
query_gpt = sample['자연어 쿼리']
query_handwritten = ''

In [3]:
# DocumentLoader 인스턴스 생성 및 문서 로드
df = pd.read_csv("./docs/test_docs.csv")

document_loader = DocumentLoader(df)
documents = document_loader.load_documents()

# LongformerTokenizer 인스턴스 생성
tokenizer = LongformerTokenizer(model_name='severinsimmler/xlm-roberta-longformer-base-16384')

# BM25Retriever 인스턴스 생성
retriever = BM25Retriever(documents, tokenizer)


sample = df.iloc[4]
query_gpt = sample['자연어 쿼리']
query_handwritten= """어젯밤에 제가 퇴근하고 동네에서 친구랑 같이 술을 한 잔 했는데요. 시간이 늦어서인지 아무리 기다려도 대리가 안오더라구요.
그래서 어쩔 수 없이 제가 운전대를 잡고, 후면주차시켰던 차를 뺴려는데, 하필 그때 차 뒤에 저랑 같이 술을 먹던 친구가 있던거에요...
제가 차를 조금 급하게 빼던 중이라서 친구가 차에 좀 세게 치였고, 지금은 병원에서 입원 중입니다.
이런 경우에도 제가 처벌받을 수 있나요? 친구인데 어떻게 좀 안될까요?"""


# 문서 검색
sorted_indices, bm25_scores = retriever.retrieve(query_handwritten)
sorted_indices, bm25_scores = retriever.retrieve(query_gpt)

In [4]:
retriever.get_search_history()

[{'query': '어젯밤에 제가 퇴근하고 동네에서 친구랑 같이 술을 한 잔 했는데요. 시간이 늦어서인지 아무리 기다려도 대리가 안오더라구요.\n그래서 어쩔 수 없이 제가 운전대를 잡고, 후면주차시켰던 차를 뺴려는데, 하필 그때 차 뒤에 저랑 같이 술을 먹던 친구가 있던거에요...\n제가 차를 조금 급하게 빼던 중이라서 친구가 차에 좀 세게 치였고, 지금은 병원에서 입원 중입니다.\n이런 경우에도 제가 처벌받을 수 있나요? 친구인데 어떻게 좀 안될까요?',
  'results': [{'index': 3,
    'document': '1. 피고인 A\r\n\r\n가. 도로교통법위반(무면허운전)\r\n\r\n피고인은 2019. 6. 27. 09:20경 자동차운전면허를 받지 아니하고 이천시 갈산동 이하 장소를 알 수 없는 도로에서부터 용인시 기흥구 C에 있는 D 아울렛 앞 교차로에 이르기까지 약 45km 구간에서 E 마이티 덤프트럭을 운전하였다.\r\n\r\n나. 교통사고처리특례법위반(치상), 도로교통법위반\r\n\r\n피고인은 E 마이티 덤프트럭의 운전업무에 종사하는 사람이다.\r\n\r\n피고인은 2019. 6. 27. 09:20경 자동차운전면허를 받지 아니하고 위 덤프트럭을 운전하여 용인시 기흥구 영덕동에 있는 영통고가 3차로를 따라 영통고가 수원 톨게이트 방면에서 수원남부서 방면으로 진행하게 되었다.\r\n\r\n그곳은 내리막길이고 내리막길이 끝나는 지점에 신호등이 설치된 교차로가 있으므로 운전업무에 종사하는 사람에게는 전방 및 좌우를 잘 살피고 제동장치 및 조향장치를 정확하게 조작하여 안전하게 운전하여 사고를 미연에 방지하여야 할 업무상 주의의무가 있었다.\r\n\r\n그럼에도 피고인은 이를 게을리 한 채 진행하던 중 내리막길이 끝나는 지점에 있는 교차로에서 신호대기로 정차 내지 서행 중인 차량들을 뒤늦게 발견하고 급제동을 한 과실로 위 덤프트럭 앞부분으로 2차로에 정차 중인 피해자 F(39세)이 운전하는 G레이 승용차 우측 뒷부분을 충돌하여 위 레이 

In [48]:
print(retriever.get_search_history()[0]['results'][0])
print(retriever.get_search_history()[0]['results'][0]['document'])

{'index': 3, 'document': '1. 피고인 A\r\n\r\n가. 도로교통법위반(무면허운전)\r\n\r\n피고인은 2019. 6. 27. 09:20경 자동차운전면허를 받지 아니하고 이천시 갈산동 이하 장소를 알 수 없는 도로에서부터 용인시 기흥구 C에 있는 D 아울렛 앞 교차로에 이르기까지 약 45km 구간에서 E 마이티 덤프트럭을 운전하였다.\r\n\r\n나. 교통사고처리특례법위반(치상), 도로교통법위반\r\n\r\n피고인은 E 마이티 덤프트럭의 운전업무에 종사하는 사람이다.\r\n\r\n피고인은 2019. 6. 27. 09:20경 자동차운전면허를 받지 아니하고 위 덤프트럭을 운전하여 용인시 기흥구 영덕동에 있는 영통고가 3차로를 따라 영통고가 수원 톨게이트 방면에서 수원남부서 방면으로 진행하게 되었다.\r\n\r\n그곳은 내리막길이고 내리막길이 끝나는 지점에 신호등이 설치된 교차로가 있으므로 운전업무에 종사하는 사람에게는 전방 및 좌우를 잘 살피고 제동장치 및 조향장치를 정확하게 조작하여 안전하게 운전하여 사고를 미연에 방지하여야 할 업무상 주의의무가 있었다.\r\n\r\n그럼에도 피고인은 이를 게을리 한 채 진행하던 중 내리막길이 끝나는 지점에 있는 교차로에서 신호대기로 정차 내지 서행 중인 차량들을 뒤늦게 발견하고 급제동을 한 과실로 위 덤프트럭 앞부분으로 2차로에 정차 중인 피해자 F(39세)이 운전하는 G레이 승용차 우측 뒷부분을 충돌하여 위 레이 승용차가 1차로로 밀리면서 그 좌측 앞부분으로 1차로에서 정차중인 피해자 H(38세) 운전의 I 벤츠C220 승용차의 우측 뒷바퀴 부분을 충격하게 하고, 피고인이 운전하는 차량은 계속 진행하여 우측 앞부분으로 3차로에 정차 중인 피해자 J(여, 37세)가 운전하는 K 레이 승용차 좌측 뒷부분을 충격한 다음, 위 레이 승용차가 밀리면서 같은 차로 앞에 정차 중인 피해자 L(여, 34세)이 운전하는 M 코란도C 승용차 우측 뒷부분을 충격하게 하고, J가 운전하는 위 레이 승용차가 그 앞으로 더 

In [49]:
df.iloc[3]

id        2019고단6035 판결 [교통사고처리특례법위반(치상), 도로교통법위반, 도로교통법...
범죄 사실     1. 피고인 A\r\n\r\n가. 도로교통법위반(무면허운전)\r\n\r\n피고인은 ...
자연어 쿼리    저는 2019년 6월 27일, 무면허 상태로 덤프트럭을 운전했습니다. 이천시 갈산동...
법령의 적용    가. 피고인 A : 각 교통사고처리 특례법 제3조 제1항, 제2항 단서 제7호, 형...
Name: 3, dtype: object

In [32]:
retriever.get_search_history()[0]['results'][3]

{'index': 6,
 'document': ['▁피',
  '고',
  '인',
  ',',
  '▁B',
  ',',
  '▁C',
  '▁등',
  '은',
  '▁2022',
  '.',
  '▁9.',
  '경',
  '▁지적',
  '장애인',
  '들을',
  '▁유',
  '인',
  '하여',
  '▁그',
  '들',
  '▁',
  '명의',
  '로',
  '▁',
  '대출',
  '을',
  '▁',
  '받',
  '거나',
  '▁상품',
  '을',
  '▁임',
  '차',
  '하여',
  '▁이를',
  '▁판매',
  '하는',
  '▁등의',
  '▁방법',
  '으로',
  '▁금',
  '원을',
  '▁편',
  '취',
  '한',
  '▁다음',
  '▁이를',
  '▁나누',
  '어',
  '▁',
  '갖',
  '기로',
  '▁공',
  '모',
  '하였다',
  '.',
  '▁피',
  '고',
  '인은',
  '▁위',
  '▁공',
  '모',
  '에',
  '▁따라',
  '▁지적',
  '장애인',
  '을',
  '▁유',
  '인',
  '하고',
  ',',
  '▁지적',
  '장애인',
  '▁',
  '명의',
  '로',
  '▁임',
  '차',
  '한',
  '▁상품',
  '을',
  '▁피',
  '고',
  '인의',
  '▁주소',
  '로',
  '▁배송',
  '▁받는',
  '▁등의',
  '▁역할을',
  '▁',
  '하였다',
  '.',
  '▁1.',
  '▁사',
  '기',
  ',',
  '▁사',
  '기',
  '미',
  '수',
  '▁피',
  '고',
  '인',
  ',',
  '▁B',
  ',',
  '▁C',
  '▁등',
  '은',
  '▁2023',
  '.',
  '▁1.',
  '▁11.',
  '경',
  '▁세종',
  '특별',
  '자치',
  '시',
  '▁소재',
  '▁불',
  '상의',
  '▁PC

In [7]:
retriever.save_search_history_to_csv()